# This notebook is more for show the what and why of the code. The production code is in the `/scr` folder

### You'll want to be in the `'../International-Maritime-Piracy/notebooks'` folder to get started

In [1]:
import os
import re

import pandas as pd

In [2]:
data_directory = os.path.join('../data','RAW_ASAM.csv')

df = pd.read_csv(data_directory, skipinitialspace = True)


df.head()

,reference,date,position,navArea,subreg,hostility,victim,description,Unnamed: 8
0,2022-127,2022-04-28,"13°43'00""N \n121°02'00""E",XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ...",NaN
1,2022-124,2022-04-27,"1°16'00""N \n104°17'00""E",XI,71,Boarding,Bulk carrier,"INDONESIA On 27 April, at 0226 local time, a r...",NaN
2,2022-121,2022-04-26,"0°04'00""S \n51°06'00""W",V,24,Boarding,Bulk carrier,"BRAZIL: On 26 April, at 0400 local time, three...",NaN
3,2022-128,2022-04-26,"22°13'00""N \n91°43'00""E",VIII,63,NaN,Singapore-flagged tanker STI MAGISTER,"BANGLADESH: On 26 April, at 0001 local time, a...",NaN
4,2022-122,2022-04-25,"59°19'00""N \n10°30'00""E",I,44,Protest,Hong Kong-flagged tanker UST LUGA,"NORWAY: On the morning of 25 April, a group of...",NaN


In [3]:
df.drop(['Unnamed: 8'], axis=1, inplace=True)

In [4]:
df

,reference,date,position,navArea,subreg,hostility,victim,description
0,2022-127,2022-04-28,"13°43'00""N \n121°02'00""E",XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ..."
1,2022-124,2022-04-27,"1°16'00""N \n104°17'00""E",XI,71,Boarding,Bulk carrier,"INDONESIA On 27 April, at 0226 local time, a r..."
2,2022-121,2022-04-26,"0°04'00""S \n51°06'00""W",V,24,Boarding,Bulk carrier,"BRAZIL: On 26 April, at 0400 local time, three..."
3,2022-128,2022-04-26,"22°13'00""N \n91°43'00""E",VIII,63,NaN,Singapore-flagged tanker STI MAGISTER,"BANGLADESH: On 26 April, at 0001 local time, a..."
4,2022-122,2022-04-25,"59°19'00""N \n10°30'00""E",I,44,Protest,Hong Kong-flagged tanker UST LUGA,"NORWAY: On the morning of 25 April, a group of..."
...,...,...,...,...,...,...,...,...
8614,1985-18,1980-01-11,"33°60'00""N \n33°60'00""E",III,56,PIRATES,INSTERGENERAL R. GUMUSPALA,"Instergeneral R. Gumuspala, a Turkish-flag gen..."
8615,1985-4,1979-10-10,"16°60'00""N \n112°00'00""E",XI,93,NaN,NaN,"SOUTH CHINA SEA, PARACEL ISLANDS. ..."
8616,1985-3,1979-02-24,"19°00'00""N \n108°00'00""E",XI,93,NaN,NaN,GULF OF TONKIN. ...
8617,1985-2,1978-05-16,"7°60'00""N \n119°60'00""E",XI,92,PIRATES,MERCHANT VESSEL,"SULU SEA: Lloyd's List of 15 May reports a 5,0..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8619 entries, 0 to 8618
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   reference    8619 non-null   object
 1   date         8619 non-null   object
 2   position     8619 non-null   object
 3   navArea      8616 non-null   object
 4   subreg       8619 non-null   int64 
 5   hostility    8107 non-null   object
 6   victim       8327 non-null   object
 7   description  8613 non-null   object
dtypes: int64(1), object(7)
memory usage: 538.8+ KB


### Split postion into Lat/Lon in prep for conversion to decimal degrees

In [6]:
df = df.join(df['position'].str.split(expand=True)).rename(columns={0:'latitude', 1:'longitude'})
df.drop('position', axis=1, inplace=True)

In [7]:
df.head(1)

,reference,date,navArea,subreg,hostility,victim,description,latitude,longitude
0,2022-127,2022-04-28,XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ...","13°43'00""N","121°02'00""E"


In [8]:
def dms2dd(field):
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', field)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd

In [9]:
df['latitude'] = df['latitude'].apply(dms2dd).round(6)
df['longitude'] = df['longitude'].apply(dms2dd).round(6)
df.head(4)

,reference,date,navArea,subreg,hostility,victim,description,latitude,longitude
0,2022-127,2022-04-28,XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ...",13.716667,121.033333
1,2022-124,2022-04-27,XI,71,Boarding,Bulk carrier,"INDONESIA On 27 April, at 0226 local time, a r...",1.266667,104.283333
2,2022-121,2022-04-26,V,24,Boarding,Bulk carrier,"BRAZIL: On 26 April, at 0400 local time, three...",-0.066667,-51.100000
3,2022-128,2022-04-26,VIII,63,NaN,Singapore-flagged tanker STI MAGISTER,"BANGLADESH: On 26 April, at 0001 local time, a...",22.216667,91.716667


### What's up with the 'hostility' column?

In [10]:
df['hostility'].value_counts()[100:150]

MILITANTS                           2
FIVE PIRATES                        2
PIRATE THEFT                        2
Three Pirates                       2
TWO SPEEDBOATS                      2
Pirate attack                       2
SMALL BOATS                         2
TUG BOAT                            2
Attempted boarding                  2
FOUR PIRATES                        2
Suspicious boat                     2
Eight robbers attempted boarding    2
ETHIOPIA                            2
CHINESE AUTHORITIES                 2
LTTE TERRORISTS                     2
CUBAN GUNBOAT                       1
THUGS                               1
PIRATESS                            1
MALDIVIAN AUTHORITIES               1
SUS[ICIOUS APPROACH                 1
YACHT                               1
PIRATES, ARMED MEN                  1
Assailants                          1
SHORESIDE PIRATES                   1
suspicous approach                  1
CHINESE BORDER VESSEL               1
SUSPIPICIOUS

### Under 'hostility' - how many different terms are there?

In [11]:
df['hostility'].nunique()

430

### Under 'victim' - how many different terms are there?

In [12]:
df['victim'].nunique()

1588

In [13]:
df['victim'].value_counts()[900:950]

AP 703                                                1
Container vessel BAHMAS                               1
AMINA                                                 1
BERTHED CHEMICAL TANKER                               1
SEA DIAMOND                                           1
ANCHORED GENERAL CARGO SHIP                           1
MOHAMED BEY                                           1
Liberia-flagged container ship ELA                    1
ARGUELLO                                              1
WATER PHOENIX                                         1
BOW ORION                                             1
QUEEN OF THE NETHERLANDS                              1
Underway oil tanker PIKE                              1
ANCHORED TANKER                                       1
BLUE COMMANDER                                        1
RIO CHARLESTON                                        1
HAI LU FENG 11                                        1
TOSNA STAR                                      

### Convert datetime to just the year for a slider

In [14]:
df['date2'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df.head(1)

,reference,date,navArea,subreg,hostility,victim,description,latitude,longitude,date2
0,2022-127,2022-04-28,XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ...",13.716667,121.033333,2022-04-28


In [15]:
df['year'] = pd.DatetimeIndex(df['date2']).year
df["date"] = pd.to_datetime(df["date2"])

df = df[['reference', 'latitude', 'longitude', 'date', 'year', 'description', 'hostility', 'victim', 'navArea', 'subreg' ]]

# sanity check
df

,reference,latitude,longitude,date,year,description,hostility,victim,navArea,subreg
0,2022-127,13.716667,121.033333,2022-04-28,2022,"PHILIPPINES: On 28 April, at 0345 local time, ...",NaN,Marshall Islands-flagged tanker SEXTANS,XI,91
1,2022-124,1.266667,104.283333,2022-04-27,2022,"INDONESIA On 27 April, at 0226 local time, a r...",Boarding,Bulk carrier,XI,71
2,2022-121,-0.066667,-51.100000,2022-04-26,2022,"BRAZIL: On 26 April, at 0400 local time, three...",Boarding,Bulk carrier,V,24
3,2022-128,22.216667,91.716667,2022-04-26,2022,"BANGLADESH: On 26 April, at 0001 local time, a...",NaN,Singapore-flagged tanker STI MAGISTER,VIII,63
4,2022-122,59.316667,10.500000,2022-04-25,2022,"NORWAY: On the morning of 25 April, a group of...",Protest,Hong Kong-flagged tanker UST LUGA,I,44
...,...,...,...,...,...,...,...,...,...,...
8614,1985-18,34.000000,34.000000,1980-01-11,1980,"Instergeneral R. Gumuspala, a Turkish-flag gen...",PIRATES,INSTERGENERAL R. GUMUSPALA,III,56
8615,1985-4,17.000000,112.000000,1979-10-10,1979,"SOUTH CHINA SEA, PARACEL ISLANDS. ...",NaN,NaN,XI,93
8616,1985-3,19.000000,108.000000,1979-02-24,1979,GULF OF TONKIN. ...,NaN,NaN,XI,93
8617,1985-2,8.000000,120.000000,1978-05-16,1978,"SULU SEA: Lloyd's List of 15 May reports a 5,0...",PIRATES,MERCHANT VESSEL,XI,92


### Right, the NaNs would mess things up. Let's convert them to something that can be parsed later

In [16]:
df.fillna("No information", inplace=True)
df.isnull().sum().sum()
df

,reference,latitude,longitude,date,year,description,hostility,victim,navArea,subreg
0,2022-127,13.716667,121.033333,2022-04-28,2022,"PHILIPPINES: On 28 April, at 0345 local time, ...",No information,Marshall Islands-flagged tanker SEXTANS,XI,91
1,2022-124,1.266667,104.283333,2022-04-27,2022,"INDONESIA On 27 April, at 0226 local time, a r...",Boarding,Bulk carrier,XI,71
2,2022-121,-0.066667,-51.100000,2022-04-26,2022,"BRAZIL: On 26 April, at 0400 local time, three...",Boarding,Bulk carrier,V,24
3,2022-128,22.216667,91.716667,2022-04-26,2022,"BANGLADESH: On 26 April, at 0001 local time, a...",No information,Singapore-flagged tanker STI MAGISTER,VIII,63
4,2022-122,59.316667,10.500000,2022-04-25,2022,"NORWAY: On the morning of 25 April, a group of...",Protest,Hong Kong-flagged tanker UST LUGA,I,44
...,...,...,...,...,...,...,...,...,...,...
8614,1985-18,34.000000,34.000000,1980-01-11,1980,"Instergeneral R. Gumuspala, a Turkish-flag gen...",PIRATES,INSTERGENERAL R. GUMUSPALA,III,56
8615,1985-4,17.000000,112.000000,1979-10-10,1979,"SOUTH CHINA SEA, PARACEL ISLANDS. ...",No information,No information,XI,93
8616,1985-3,19.000000,108.000000,1979-02-24,1979,GULF OF TONKIN. ...,No information,No information,XI,93
8617,1985-2,8.000000,120.000000,1978-05-16,1978,"SULU SEA: Lloyd's List of 15 May reports a 5,0...",PIRATES,MERCHANT VESSEL,XI,92


In [17]:
os.makedirs('../data/in_progress', exist_ok=True)  

df.to_csv('../data/in_progress/v1_lat_long.csv') 

# Future work

- [x] Remove the excess spaces in the description column.


- [X] Fix the 'reference' column values
    - [x] contact NGA and see if this would be of interest (**working**)
    

- [ ] Regularize the 'hostility' columns values - easy first pass would be to trust the data ruthlessly pare down the columns


- [ ] Regularize the 'victim' columns values  - easy first pass would be to trust the data ruthlessly pare down the columns


- [ ] NLP model to set the hostility and possibly victim columns




# <center>Remove the excess spaces in the 'description' column</center>


The excel file for the description has a lot a weird spacing. Does that show here?

* *Why, yes it does.* But it only shows if you have a wide column width

Also, another plug for `skipinitialspace = True` flag without it you need to add `df.hostility = df.hostility.str.lstrip()` 

In [18]:
pd.set_option('display.max_colwidth', None)
df['description'][df["reference"]=='1991-21']

8450    W. MEDITERRANIAN-COAST OF ALGERIA-VICINITY CAP DE TROIS FOURCHES                                                                                 07JUN91 Spanish Trawler ENMANUEL was fired upon by an Algerian      Patrol Vessel, killing crewman Jose Salmeron of Almeria, Spain.        The attack took place in International Waters, 64-miles SW Cabo        de Greta.(See also 91-0019)
Name: description, dtype: object

In [19]:
df

,reference,latitude,longitude,date,year,description,hostility,victim,navArea,subreg
0,2022-127,13.716667,121.033333,2022-04-28,2022,"PHILIPPINES: On 28 April, at 0345 local time, three robbers boarded the Marshall Islands-flagged tanker SEXTANS anchored in Batangas Anchorage, near position 13-43N 121-02E. The duty crew sighted the robbers in the forecastle deck. The alarm was raised, and the crew were mustered. The perpetrators escaped with stolen buckets of grease, a bundle of rope, and hatch butterfly nuts. The incident was reported to the local authorities.",No information,Marshall Islands-flagged tanker SEXTANS,XI,91
1,2022-124,1.266667,104.283333,2022-04-27,2022,"INDONESIA On 27 April, at 0226 local time, a robber was spotted in the engine room of a bulk carrier underway in the eastbound lane of the Singapore Strait Traffic Separation Scheme (TSS), near position 01-16N 104-17E. It was reported that the ship and all crew members were safe and nothing was stolen.",Boarding,Bulk carrier,XI,71
2,2022-121,-0.066667,-51.100000,2022-04-26,2022,"BRAZIL: On 26 April, at 0400 local time, three robbers boarded a bulk carrier anchored at Macapa Anchorage, near position 00-04S 051-06W. The duty crew member spotted the robbers. The alarm was raised, and the crew were mustered, after which the perpetrators escaped empty-handed.",Boarding,Bulk carrier,V,24
3,2022-128,22.216667,91.716667,2022-04-26,2022,"BANGLADESH: On 26 April, at 0001 local time, a group of robbers boarded the Singapore-flagged tanker STI MAGISTER anchored in the outer Chittagong Anchorage near position 22-13N 091-43E. The duty officer sighted four robbers on the forecastle deck through the vessel’s closed circuit television system. The duty officer raised the alarm and the crew mustered. After the alarm was sounded, three more robbers were seen exiting the forecastle store. Subsequently, crew members saw about eight robbers escaping into a small boat. The vessel was searched, and the forecastle store door lock was found breached and a set of accommodation ladder wire rope was missing. The incident was reported to the local authorities and the coast guard boarded the vessel to conduct an investigation.",No information,Singapore-flagged tanker STI MAGISTER,VIII,63
4,2022-122,59.316667,10.500000,2022-04-25,2022,"NORWAY: On the morning of 25 April, a group of protesters onboard small boats and kayaks came alongside the Hong Kong-flagged tanker UST LUGA while anchored at Skagerrak, near position 59-19N 010-30E. According to press reports, the group was protestinq against oil imports from Russia. Activists onboard one of the small boats chained themselves to the ship's anchor chain to prevent the ship from raising its anchor, While others in kayaks unfurled banners readinq ""oil fuels war"". The vessel was loaded with 95,000 tons of crude oil from the port of Ust-Luga, Russia. The local police and coast guard came to monitor the situation. Reports indicate that the vessel later berthed at the port of Slagen at 1800 local time the same day.",Protest,Hong Kong-flagged tanker UST LUGA,I,44
...,...,...,...,...,...,...,...,...,...,...
8614,1985-18,34.000000,34.000000,1980-01-11,1980,"Instergeneral R. Gumuspala, a Turkish-flag general cargo vessel, was intercepted by Corvettes and shot at with machine guns 24 miles west of Beirut.",PIRATES,INSTERGENERAL R. GUMUSPALA,III,56
8615,1985-4,17.000000,112.000000,1979-10-10,1979,"SOUTH CHINA SEA, PARACEL ISLANDS. Mariners transiting the waters in and around Paracel Islands are advised of the possibility of unusual, or abnormal activities due to competing jurisdictional claims which could possibly lead to hostilities.",No information,No information,XI,93
8616,1985-3,19.000000,108.000000,1979-02-24,1979,GULF OF TONKIN. Mariners should be advised that an uncertain situation exists in the Gulf of Tonkin and waters adjacent to Hainan Island due to increased tensions relative to the China/Viet Nam border conflict.,No information,No information,

In [20]:
df.replace('\s+', ' ',regex=True,inplace=True)

In [21]:
# OK, that seems to have done it.
#df

In [22]:
df.groupby(pd.Grouper(key="date")).mean()
# gp = gp.divide(gp.sum(axis=1), axis=0).multiply(100)
df

,reference,latitude,longitude,date,year,description,hostility,victim,navArea,subreg
0,2022-127,13.716667,121.033333,2022-04-28,2022,"PHILIPPINES: On 28 April, at 0345 local time, three robbers boarded the Marshall Islands-flagged tanker SEXTANS anchored in Batangas Anchorage, near position 13-43N 121-02E. The duty crew sighted the robbers in the forecastle deck. The alarm was raised, and the crew were mustered. The perpetrators escaped with stolen buckets of grease, a bundle of rope, and hatch butterfly nuts. The incident was reported to the local authorities.",No information,Marshall Islands-flagged tanker SEXTANS,XI,91
1,2022-124,1.266667,104.283333,2022-04-27,2022,"INDONESIA On 27 April, at 0226 local time, a robber was spotted in the engine room of a bulk carrier underway in the eastbound lane of the Singapore Strait Traffic Separation Scheme (TSS), near position 01-16N 104-17E. It was reported that the ship and all crew members were safe and nothing was stolen.",Boarding,Bulk carrier,XI,71
2,2022-121,-0.066667,-51.100000,2022-04-26,2022,"BRAZIL: On 26 April, at 0400 local time, three robbers boarded a bulk carrier anchored at Macapa Anchorage, near position 00-04S 051-06W. The duty crew member spotted the robbers. The alarm was raised, and the crew were mustered, after which the perpetrators escaped empty-handed.",Boarding,Bulk carrier,V,24
3,2022-128,22.216667,91.716667,2022-04-26,2022,"BANGLADESH: On 26 April, at 0001 local time, a group of robbers boarded the Singapore-flagged tanker STI MAGISTER anchored in the outer Chittagong Anchorage near position 22-13N 091-43E. The duty officer sighted four robbers on the forecastle deck through the vessel’s closed circuit television system. The duty officer raised the alarm and the crew mustered. After the alarm was sounded, three more robbers were seen exiting the forecastle store. Subsequently, crew members saw about eight robbers escaping into a small boat. The vessel was searched, and the forecastle store door lock was found breached and a set of accommodation ladder wire rope was missing. The incident was reported to the local authorities and the coast guard boarded the vessel to conduct an investigation.",No information,Singapore-flagged tanker STI MAGISTER,VIII,63
4,2022-122,59.316667,10.500000,2022-04-25,2022,"NORWAY: On the morning of 25 April, a group of protesters onboard small boats and kayaks came alongside the Hong Kong-flagged tanker UST LUGA while anchored at Skagerrak, near position 59-19N 010-30E. According to press reports, the group was protestinq against oil imports from Russia. Activists onboard one of the small boats chained themselves to the ship's anchor chain to prevent the ship from raising its anchor, While others in kayaks unfurled banners readinq ""oil fuels war"". The vessel was loaded with 95,000 tons of crude oil from the port of Ust-Luga, Russia. The local police and coast guard came to monitor the situation. Reports indicate that the vessel later berthed at the port of Slagen at 1800 local time the same day.",Protest,Hong Kong-flagged tanker UST LUGA,I,44
...,...,...,...,...,...,...,...,...,...,...
8614,1985-18,34.000000,34.000000,1980-01-11,1980,"Instergeneral R. Gumuspala, a Turkish-flag general cargo vessel, was intercepted by Corvettes and shot at with machine guns 24 miles west of Beirut.",PIRATES,INSTERGENERAL R. GUMUSPALA,III,56
8615,1985-4,17.000000,112.000000,1979-10-10,1979,"SOUTH CHINA SEA, PARACEL ISLANDS. Mariners transiting the waters in and around Paracel Islands are advised of the possibility of unusual, or abnormal activities due to competing jurisdictional claims which could possibly lead to hostilities.",No information,No information,XI,93
8616,1985-3,19.000000,108.000000,1979-02-24,1979,GULF OF TONKIN. Mariners should be advised that an uncertain situation exists in the Gulf of Tonkin and waters adjacent to Hainan Island due to increased tensions relative to the China/Viet Nam border conflict.,No information,No information,

# <center>Fix the 'reference' column values</center>

In [23]:
df['reference'] = (df['date'].dt.year.astype(str) + '-' + (df.sort_values('date')
                    .groupby(df['date'].dt.year).cumcount()+1).astype(str))

In [24]:
df

,reference,latitude,longitude,date,year,description,hostility,victim,navArea,subreg
0,2022-114,13.716667,121.033333,2022-04-28,2022,"PHILIPPINES: On 28 April, at 0345 local time, three robbers boarded the Marshall Islands-flagged tanker SEXTANS anchored in Batangas Anchorage, near position 13-43N 121-02E. The duty crew sighted the robbers in the forecastle deck. The alarm was raised, and the crew were mustered. The perpetrators escaped with stolen buckets of grease, a bundle of rope, and hatch butterfly nuts. The incident was reported to the local authorities.",No information,Marshall Islands-flagged tanker SEXTANS,XI,91
1,2022-113,1.266667,104.283333,2022-04-27,2022,"INDONESIA On 27 April, at 0226 local time, a robber was spotted in the engine room of a bulk carrier underway in the eastbound lane of the Singapore Strait Traffic Separation Scheme (TSS), near position 01-16N 104-17E. It was reported that the ship and all crew members were safe and nothing was stolen.",Boarding,Bulk carrier,XI,71
2,2022-112,-0.066667,-51.100000,2022-04-26,2022,"BRAZIL: On 26 April, at 0400 local time, three robbers boarded a bulk carrier anchored at Macapa Anchorage, near position 00-04S 051-06W. The duty crew member spotted the robbers. The alarm was raised, and the crew were mustered, after which the perpetrators escaped empty-handed.",Boarding,Bulk carrier,V,24
3,2022-111,22.216667,91.716667,2022-04-26,2022,"BANGLADESH: On 26 April, at 0001 local time, a group of robbers boarded the Singapore-flagged tanker STI MAGISTER anchored in the outer Chittagong Anchorage near position 22-13N 091-43E. The duty officer sighted four robbers on the forecastle deck through the vessel’s closed circuit television system. The duty officer raised the alarm and the crew mustered. After the alarm was sounded, three more robbers were seen exiting the forecastle store. Subsequently, crew members saw about eight robbers escaping into a small boat. The vessel was searched, and the forecastle store door lock was found breached and a set of accommodation ladder wire rope was missing. The incident was reported to the local authorities and the coast guard boarded the vessel to conduct an investigation.",No information,Singapore-flagged tanker STI MAGISTER,VIII,63
4,2022-110,59.316667,10.500000,2022-04-25,2022,"NORWAY: On the morning of 25 April, a group of protesters onboard small boats and kayaks came alongside the Hong Kong-flagged tanker UST LUGA while anchored at Skagerrak, near position 59-19N 010-30E. According to press reports, the group was protestinq against oil imports from Russia. Activists onboard one of the small boats chained themselves to the ship's anchor chain to prevent the ship from raising its anchor, While others in kayaks unfurled banners readinq ""oil fuels war"". The vessel was loaded with 95,000 tons of crude oil from the port of Ust-Luga, Russia. The local police and coast guard came to monitor the situation. Reports indicate that the vessel later berthed at the port of Slagen at 1800 local time the same day.",Protest,Hong Kong-flagged tanker UST LUGA,I,44
...,...,...,...,...,...,...,...,...,...,...
8614,1980-1,34.000000,34.000000,1980-01-11,1980,"Instergeneral R. Gumuspala, a Turkish-flag general cargo vessel, was intercepted by Corvettes and shot at with machine guns 24 miles west of Beirut.",PIRATES,INSTERGENERAL R. GUMUSPALA,III,56
8615,1979-2,17.000000,112.000000,1979-10-10,1979,"SOUTH CHINA SEA, PARACEL ISLANDS. Mariners transiting the waters in and around Paracel Islands are advised of the possibility of unusual, or abnormal activities due to competing jurisdictional claims which could possibly lead to hostilities.",No information,No information,XI,93
8616,1979-1,19.000000,108.000000,1979-02-24,1979,GULF OF TONKIN. Mariners should be advised that an uncertain situation exists in the Gulf of Tonkin and waters adjacent to Hainan Island due to increased tensions relative to the China/Viet Nam border conflict.,No information,No information,X

# <center>Regularize the 'hostility' columns values</center>

This is tricky and I think and NLP approach will be the better overall but in terms of getting something out the door, I think this is fine.

In [25]:
def hostilities(df):
    h = df.hostility.unique()
    h.sort()
    print(h)

def body_count(df):
    return df.hostility.nunique()

In [26]:
first_cut = body_count(df)
first_cut

431

In [27]:
hostilities(df)

['15 PIRATES' '24-APR-2002' '25 SPEEDBOATS' 'ACTIVISTS'
 'ALGERIAN COASTAL PATROL' 'ALLEGED POLISARIO FRONT' 'AMERBER II'
 'ARMED PIRATES' 'ARMED ROBBERS' 'ARMED ROBBERY' 'ASSAILANTS' 'ATTACKER'
 'ATTACKERS' 'ATTEMPTED BOARDING' 'AUTHORITIES IN JACMEL, HAITI'
 'Algerian Patrol Vessel' 'Approached by fishing boat with four armed men'
 'Approached by skiff' 'Armed Pirates' 'Armed boarders'
 'Armed men in a speedboat' 'Armed persons boarding'
 'Armed persons in a skiff fired upon a tanker'
 'Armed robbers boarded vessel' 'Armed robbers boarding' 'Armed robebrs'
 'Assailants' 'Attacked and boarded' 'Attacked and fired upon'
 'Attacked three passenger boats' 'Attempted Boarding'
 'Attempted boarding' 'Attempted robbery' 'BANDITS' 'BLUE SPEEDBOAT'
 'BOARDED BY ARMED ROBBERS' 'BOARDER' 'BOARDERS' 'BOARDING' 'BOAT'
 'BURMESE PIRATES' 'Bandits' 'Boarded by five robbers' 'Boarders'
 'Boarding' 'Boarding and attempted robbery' 'Boarding by robbers'
 'Boarding by three robbers' 'Boarding vessel' '

### Nobody can spell ~~SIUSPICIOUS~~, ~~suspicous~~, ~~SUS\[ICIOUS~~, ~~SUSPIPICIOUS~~ suspicious

In [28]:
#df[df["hostility"].str.contains(r"^Su(s|c)(p|])", regex=True, case=False)]

df.hostility = df.hostility.str.lstrip()
df['hostility'] = df['hostility'].str.replace(r'(?i)^Su[sc][][p][ie].*', 'Suspicious Approach', regex=True)

In [29]:
second_cut = body_count(df)
print(f" This took us from {first_cut} to {second_cut}")

 This took us from 431 to 398


### Protestors and Stoaways seem simple

In [30]:
df['hostility'] = df['hostility'].str.replace(r'(?i)Protest.*', 'Protest', regex=True)
df['hostility'] = df['hostility'].str.replace(r'(?i)Stow.*', 'Stowaway', regex=True)

In [31]:
third_cut = body_count(df)
print(f" This took us from {second_cut} to {third_cut}")

 This took us from 398 to 393


### A whole lot of robberies to take care of

Note:
Creating an extra column:

```
df['Robbery?'] = df["hostility"].str.contains("robbe*", case=False, na=False).astype(int)
``` 
followed by:
```
df.loc[df["hostility"].str.contains("robbe", case=False, na=False)] = "OMG! A ROBBERY!"
```
was a good way to make sure to step though things.

In [32]:
88888888 ----------->>>  stopping point  <<<------------- 8888888

SyntaxError: invalid syntax (1658041875.py, line 1)

In [ ]:
df.loc[df["hostility"].str.contains("robbe", case=False, na=False)] = "Robbery"

Oddly enough there is no 'Piracy' category in a database on pirates. The categories are:

- Attempted Boarding – Close approach or hull-to-hull contact with report that boarding
paraphernalia were employed or visible in the approaching boat.
- Blocking – Hampering safe navigation, docking, or undocking of a vessel as a means of protest.
- Boarding – Unauthorized embarkation of a vessel by persons not part of its complement without
successfully taking control of the vessel.
- Fired Upon – Weapons discharged at or toward a vessel.
- Hijacking – Unauthorized seizure and retention of a vessel by persons not part of its complement.
- Kidnapping – Unauthorized forcible removal of persons belonging to the vessel from it.
- Hijacking/Kidnapping Combination – Unauthorized seizure and retention of a vessel by persons not
- Robbery – Theft from a vessel or from persons aboard the vessel.
- Suspicious Approach 

In [ ]:
df.loc[df["hostility"].str.contains("Pirate*", case=False, na=False)] = "OOOOHHHH Pirates!"

In [ ]:
df['hostility'].nunique()

In [ ]:
os.makedirs('../data/in_progress', exist_ok=True)  

df.to_csv('../data/in_progress/v1_lat_long.csv') 